In [100]:
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
import re
import string
df0=pd.read_csv("data.csv")
df = pd.read_csv("data_2.csv")
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   6040 non-null   int64 
 1   TEXT    6040 non-null   object
dtypes: int64(1), object(1)
memory usage: 94.5+ KB


[nltk_data] Downloading package stopwords to /home/srj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.WordNetLemmatizer()

In [102]:
def clean_text(text):
    no_punctuation = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    stems = [ps.lemmatize(word) for word in tokens if word not in stopwords] # Remove Stopwords
    return stems

In [103]:
print(sorted(Counter(df['LABEL']).items()))

[(0, 4846), (1, 4893), (2, 4950)]


In [104]:
X=df["TEXT"].astype(str).str.lower()
Y=df["LABEL"]

In [105]:
#X = [str(x) for x in X] #converting to iterable object before passing to fit transform
#vect = TfidfVectorizer(analyzer=clean_text)
#vector=vect.fit_transform(X)
#X=pd.DataFrame(vector.toarray())

In [106]:
split_index = 6042
X_train, X_test, Y_train, Y_test = train_test_split(X[:split_index], Y[:split_index],random_state=42)

In [107]:
X_train = X[split_index:]
Y_train = Y[split_index:]

In [108]:
print(X.head())

0    your opinion about me? 1. over 2. jada 3. kusr...
1    what's up? do you want me to come online? if y...
2                         so u workin overtime nigpun?
3    also sir, i sent you an email about how to log...
4    please stay at home. to encourage the notion o...
Name: TEXT, dtype: object


In [109]:
vectorizer=TfidfVectorizer()
X_train_tfidf=vectorizer.fit_transform(X_train)
clf=LogisticRegression()
clf.fit(X_train_tfidf, Y_train)

LogisticRegression()

In [110]:
X_test_tfidf = vectorizer.transform(X_test)
y_pred = clf.predict(X_test_tfidf)
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 18.00%
